# ✅ Environment Setting

In [3]:
!wget https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar

--2023-05-24 08:47:40--  https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870644 (850K) [application/java-archive]
Saving to: ‘hadoop-aws-3.3.1.jar.1’

hadoop-aws-3.3.1.ja 100%[===================>] 850.24K  --.-KB/s    in 0.01s   

2023-05-24 08:47:41 (85.0 MB/s) - ‘hadoop-aws-3.3.1.jar.1’ saved [870644/870644]

--2023-05-24 08:47:46--  https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.1/hadoop-aws-3.3.1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209, 2a04:4e42:4c::209, ...
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 870644 (850K) [application/java-archive]
Saving to:

In [4]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages "org.apache.hadoop:hadoop-aws:3.3.1" pyspark-shell'

# ✅ SparkSession 생성 및 데이터 불러오기

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Read from S3") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.1") \
    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.access.key","(access_key)") \
    .config("spark.hadoop.fs.s3a.secret.key","(secret_key)") \
    .getOrCreate()


In [7]:
# # s3 bucket에서 데이터 읽어오기 (10개 웹툰)
# df = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .json("s3a://(버킷명)/{805691,805893,805437,570506,644112,648419,64997,671421,676695,687915}/*.json")

In [76]:
# s3 bucket에서 데이터 읽어오기 (5개 웹툰)
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .json("s3a://(버킷명)/{805691,805893,805437,804862,804868}/*.json")

In [44]:
# # s3 bucket에서 데이터 읽어오기 (2개 웹툰)
# df = spark.read \
#     .option("header", "true") \
#     .option("inferSchema", "true") \
#     .json("s3a://(버킷명)/{805893,805437}/*.json")

> ## 총 레코드 수 (best 댓글 수)

In [77]:
df.count()

815

In [78]:
# 스키마 형태
df.printSchema()

root
 |-- comment: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- epi_no: long (nullable = true)
 |-- is_best: boolean (nullable = true)
 |-- login_id: string (nullable = true)
 |-- nickname: string (nullable = true)
 |-- recomm_cnt: string (nullable = true)
 |-- reply_cnt: string (nullable = true)
 |-- save_date: string (nullable = true)
 |-- title_id: string (nullable = true)
 |-- unrecomm_cnt: string (nullable = true)
 |-- write_date: string (nullable = true)



In [79]:
df.show()

+-------------------------------------+----------+------+-------+--------+------------------+----------+---------+-----------------+--------+------------+--------------------+
|                              comment|comment_id|epi_no|is_best|login_id|          nickname|recomm_cnt|reply_cnt|        save_date|title_id|unrecomm_cnt|          write_date|
+-------------------------------------+----------+------+-------+--------+------------------+----------+---------+-----------------+--------+------------+--------------------+
|   결국 낙하산은 낙하산으로 끝내는...| 452407213|    13|   true|jhw0****|아꽁치조림먹고싶다|       810|        5|1683147027.928622|  805893|           3|2023-03-24T03:03:...|
|    이 웹툰은 로맨스 중심으로 돌아...| 453760787|    13|   true|pril****|              프릴|       285|        0|1683147030.885027|  805893|           1|2023-04-27T23:12:...|
|   그래도 주연은 아니라도 비중있는...| 453493549|    13|   true|seoj****|               seo|       205|        0|1683147032.881402|  805893|           2|2023-04-21T10:39:.

In [80]:
# 웹툰 별 댓글 개수 
from pyspark.sql.functions import count

comments_cnt = df.groupBy("title_id").agg(
    count("epi_no").alias("epi_no_cnt")
    )

In [81]:
comments_cnt.show()

+--------+----------+
|title_id|epi_no_cnt|
+--------+----------+
|  805437|       192|
|  804862|       228|
|  804868|       189|
|  805893|       113|
|  805691|        93|
+--------+----------+



In [82]:
# # DataFrame의 모든 내용 확인
# rows = df.collect()
# for row in rows:
#     print(row)


# ️✅ 분석: 답글(unrecomm_cnt) 수 

#### 사전작업

In [83]:
from pyspark.sql.functions import col, avg, expr, desc

In [84]:
df = df.withColumn("reply_cnt", col("reply_cnt").cast("integer"))

> ## 1. 답글 수의 평균

>> ### 1) 전체 답글 수의 평균





In [85]:
avg_reply_cnt = df.select(
    avg("reply_cnt").alias("avg_reply_cnt")
).selectExpr(
    "avg_reply_cnt"
)

In [86]:
avg_reply_cnt.show()

+-----------------+
|    avg_reply_cnt|
+-----------------+
|5.743558282208589|
+-----------------+



>> ### 2)  답글 수의 평균이 높은 웹툰 Top10

In [87]:
top10_reply_by_toon= df.groupBy("title_id").agg(
                            avg("reply_cnt").alias("avg_reply_cnt")
                        ).orderBy(desc("avg_reply_cnt")) \
                        .limit(10)

In [88]:
top10_reply_by_toon.show()

+--------+------------------+
|title_id|     avg_reply_cnt|
+--------+------------------+
|  804862|13.833333333333334|
|  805691| 3.086021505376344|
|  804868|2.8095238095238093|
|  805437|          2.359375|
|  805893| 2.265486725663717|
+--------+------------------+



>> ### 3) 답글 수의 평균이 높은 회차 Top 10

In [89]:
top10_reply_by_epi = df.groupBy("epi_no").agg(
                  avg("reply_cnt").alias("avg_reply_cnt")
                  ).orderBy(desc("avg_reply_cnt")) \
                  .limit(10)

In [90]:
top10_reply_by_epi.show()

+------+-----------------+
|epi_no|    avg_reply_cnt|
+------+-----------------+
|    18|             21.0|
|     1|8.266666666666667|
|     5|          8.21875|
|     6|7.979591836734694|
|    12|7.254901960784314|
|     7| 6.74468085106383|
|     3|6.714285714285714|
|    17|5.608695652173913|
|     4|             4.95|
|    16|4.882352941176471|
+------+-----------------+



> ## 2. 답글 수 분포

>> ### 1) 답글 수 Top 10

In [91]:
top_10_records = df.orderBy(col("reply_cnt").desc()).limit(10)

In [92]:
top_10_records.show()

+----------------------------------+----------+------+-------+--------+--------+----------+---------+-----------------+--------+------------+--------------------+
|                           comment|comment_id|epi_no|is_best|login_id|nickname|recomm_cnt|reply_cnt|        save_date|title_id|unrecomm_cnt|          write_date|
+----------------------------------+----------+------+-------+--------+--------+----------+---------+-----------------+--------+------------+--------------------+
|  님들 근데 이거 웹툰 뭐라고 읽...| 451512366|    12|   true|pkp8****|    태성|      1104|      149|1684514913.300666|  804862|          18|2023-03-01T00:48:...|
| 이거 드라마로 만들어서 원빈이 ...| 449696399|     6|   true|bomi****|    보민|      3969|      114|1684514373.418594|  804862|         191|2023-01-18T00:40:...|
|    월:퀘지주화:김부장 수:백XX ...| 449678801|     1|   true|jswo****|   white|      8534|      111| 1684514027.73187|  804862|         100|2023-01-17T22:13:...|
|   캐슬 따라한 것만 같달까.. 최...| 450055677|     7|   true|hwam****| 

# ️✅ 분석: '댓글'의 좋아요/싫어요 수 

사전 작업

In [93]:
df = df.withColumn("recomm_cnt", col("recomm_cnt").cast("integer"))
df = df.withColumn("unrecomm_cnt", col("unrecomm_cnt").cast("integer"))

> ## 0. 좋아요/싫어요 수 분포

>> ### 1) 좋아요/싫어요 수 Top 10

In [94]:
top_10_recomm = df.orderBy(col("recomm_cnt").desc()).limit(10)
top_10_unrecomm = df.orderBy(col("unrecomm_cnt").desc()).limit(10)

In [95]:
top_10_recomm.show()

+----------------------------------+----------+------+-------+--------+------------+----------+---------+-----------------+--------+------------+--------------------+
|                           comment|comment_id|epi_no|is_best|login_id|    nickname|recomm_cnt|reply_cnt|        save_date|title_id|unrecomm_cnt|          write_date|
+----------------------------------+----------+------+-------+--------+------------+----------+---------+-----------------+--------+------------+--------------------+
| 아니 에바잖아 지금 죽기엔 너무...| 449713451|     4|   true|aiub****|        파도|     11071|       33|1684514187.471901|  804862|          45|2023-01-18T12:54:...|
|  태준이형 이젠 수요일 1등도 하...| 449678286|     1|   true|jswo****|       white|     10802|       53|1684514024.895106|  804862|          91|2023-01-17T22:03:...|
|   진짜 죽은 거야? 내 최애된지 ...| 449689770|     5|   true|clai****|      하련솔|      9663|       47|1684514309.224483|  804862|          23|2023-01-17T23:29:...|
| 근데 개쳐발린거 보고 ㅈㄴ 건방...| 450291622|     

In [96]:
top_10_unrecomm.show()

+---------------------------------+----------+------+-------+--------+------------+----------+---------+-----------------+--------+------------+--------------------+
|                          comment|comment_id|epi_no|is_best|login_id|    nickname|recomm_cnt|reply_cnt|        save_date|title_id|unrecomm_cnt|          write_date|
+---------------------------------+----------+------+-------+--------+------------+----------+---------+-----------------+--------+------------+--------------------+
|이거 드라마로 만들어서 원빈이 ...| 449696399|     6|   true|bomi****|        보민|      3969|      114|1684514373.418594|  804862|         191|2023-01-18T00:40:...|
|   작가님 저희가 10시30분에 올...| 451182092|    11|   true|maca****|         mac|      2845|       25| 1684514802.42765|  804862|         149|2023-02-21T22:35:...|
|캐슬이 더 무개감있다.캐슬은 넘...| 452323755|    15|   true|dlwj****|         dlw|      1556|       33|1684515157.294156|  804862|         126|2023-03-21T23:26:...|
|   머리타령좀 그만했으면.........| 450519474|     9

> ## 1. 좋아요/싫어요 수의 평균



>> ### 1) 전체 좋아요/싫어요 수의 평균



In [97]:
avg_recomm_cnt = df.select(
    avg("recomm_cnt").alias("avg_recomm_cnt")
).selectExpr(
    "avg_recomm_cnt"
)

avg_unrecomm_cnt = df.select(
    avg("unrecomm_cnt").alias("avg_unrecomm_cnt")
).selectExpr(
    "avg_unrecomm_cnt"
)

In [98]:
avg_recomm_cnt.show()

+----------------+
|  avg_recomm_cnt|
+----------------+
|857.198773006135|
+----------------+



In [99]:
avg_unrecomm_cnt.show()

+------------------+
|  avg_unrecomm_cnt|
+------------------+
|13.231901840490798|
+------------------+



>>  ### 2) 좋아요/싫어요 수의 평균이 높은 웹툰 Top10

In [100]:
top10_recomm_by_toon= df.groupBy("title_id").agg(
                            avg("recomm_cnt").alias("avg_recomm_cnt")
                        ).orderBy(desc("avg_recomm_cnt")) \
                        .limit(10)

top10_unrecomm_by_toon= df.groupBy("title_id").agg(
                            avg("unrecomm_cnt").alias("avg_unrecomm_cnt")
                        ).orderBy(desc("avg_unrecomm_cnt")) \
                        .limit(10)

In [101]:
top10_recomm_by_toon.show()

+--------+------------------+
|title_id|    avg_recomm_cnt|
+--------+------------------+
|  804862|1537.5657894736842|
|  804868| 960.2380952380952|
|  805893| 531.7433628318585|
|  805437|        450.296875|
|  805691|215.30107526881721|
+--------+------------------+



In [102]:
top10_unrecomm_by_toon.show()

+--------+------------------+
|title_id|  avg_unrecomm_cnt|
+--------+------------------+
|  804862|31.666666666666668|
|  805437| 8.276041666666666|
|  804868| 6.634920634920635|
|  805691| 5.172043010752688|
|  805893|2.1238938053097347|
+--------+------------------+



>> ### 3) 좋아요/싫어요 수의 평균이 높은 회차 Top 10

In [103]:
top10_recomm_by_epi = df.groupBy("epi_no").agg(
                  avg("recomm_cnt").alias("avg_recomm_cnt")
                  ).orderBy(desc("avg_recomm_cnt")) \
                  .limit(10)

top10_unrecomm_by_epi = df.groupBy("epi_no").agg(
                  avg("unrecomm_cnt").alias("avg_unrecomm_cnt")
                  ).orderBy(desc("avg_unrecomm_cnt")) \
                  .limit(10)

In [104]:
top10_recomm_by_epi.show()

+------+------------------+
|epi_no|    avg_recomm_cnt|
+------+------------------+
|    18|2121.4545454545455|
|     1|           1346.15|
|     5|        1345.21875|
|     4|            1220.0|
|     3|1057.4489795918366|
|     2| 902.8222222222222|
|     7| 866.8085106382979|
|    15| 799.8461538461538|
|     8|             788.4|
|    17| 740.9565217391304|
+------+------------------+



In [105]:
top10_unrecomm_by_epi.show()

+------+------------------+
|epi_no|  avg_unrecomm_cnt|
+------+------------------+
|    18|24.545454545454547|
|     1|18.716666666666665|
|     5|          18.09375|
|     9|16.280701754385966|
|    10|15.055555555555555|
|     6|14.938775510204081|
|    17|14.565217391304348|
|     8|              13.6|
|     2| 13.11111111111111|
|     3| 12.89795918367347|
+------+------------------+



# ️✅ 분석: 댓글의 길이

## 0. 전체 댓글의 길이의 평균

In [106]:
from pyspark.sql.functions import avg, length

avg_comment_length = df.select(avg(length("comment")).alias("avg_comment_length"))

In [107]:
avg_comment_length.show()

+------------------+
|avg_comment_length|
+------------------+
|40.750920245398774|
+------------------+

